In [3]:
np.__version__

'1.18.1'

In [8]:
!python3 -m pip install cmake -U

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable


In [1]:
!python3 -m pip install tensorflow-gpu==2.1.0 tensorflow-probability -U

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow_probability-0.19.0-py2.py3-none-any.whl (6.7 MB)
  Using cached tensorflow_probability-0.18.0-py2.py3-none-any.whl (6.6 MB)
  Using cached tensorflow_probability-0.17.0-py2.py3-none-any.whl (6.5 MB)
  Using cached tensorflow_probability-0.16.0-py2.py3-none-any.whl (6.3 MB)
  Using cached tensorflow_probability-0.15.0-py2.py3-none-any.whl (5.7 MB)
  Using cached tensorflow_probability-0.14.1-py2.py3-none-any.whl (5.7 MB)
  Using cached tensorflow_probability-0.14.0-py2.py3-none-any.whl (5.7 MB)
  Usi

In [12]:
!python3 -m pip install keras==2.2.4

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


___

In [1]:
import tensorflow as tf
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

tf.compat.v1.experimental.output_all_intermediates(True)

In [2]:
import numpy as np

In [3]:
from cvae_models import *

In [4]:
X = np.random.normal(size=(3, 500))
y = np.random.normal(size=(3, 433))

___

In [6]:
emb_model = get_emb_model(in_dim=500, out_dim=433)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
emb_model.fit(X, y, batch_size=1, epochs=1)

Train on 1 samples
1/1 [==============================] - 0s 416ms/sample - loss: 0.8139 - acc: 0.0000e+00


In [8]:
emb_model.save('emb.h5')

___

In [24]:
gauss_model, decoder = get_gauss_model(trained_emb_model_path='emb.h5', in_dim=500, out_dim=433, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: input_11:0


In [25]:
gauss_model.fit([X, y], 
                [y[:, :3], y[:, 3:7], 
                y[:, 7:129+7], y[:, 129+7:]], 
                batch_size=3, epochs=1)

Train on 3 samples
3/3 [==============================] - 1s 392ms/sample - loss: -20.3199 - model_18_loss: 0.8060 - model_18_1_loss: 2.2822 - model_18_2_loss: 5.0991e-19 - model_18_3_loss: -23.4163


In [26]:
top_ans = generate_gauss(X, decoder, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3, s=100)

100/100

In [41]:
for i in range(len(top_ans)):
    top1_t = top_ans[i][0][0]
    top1_p = top_ans[i][0][1]
    top1_adds = top_ans[i][0][2:-1]
    top1_catalyst = top_ans[i][0][-1]

___

In [5]:
rnf_model, decoder = get_rnf_gauss_model(trained_emb_model_path='emb.h5', in_dim=500, out_dim=433, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: input_1:0


In [6]:
fit_rnf_gauss_model(rnf_model, epochs=1, batch_size=3, X_train=X, y_train=y, t_dim=3, p_dim=4, add_dim=129)

-------------------------------------------------------------
1
-------------------------------------------------------------


/home/cyberdan/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/cyberdan/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/home/cyberdan/.local/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 3 samples
3/3 [==============================] - 3s 886ms/sample - loss: -6.6122 - model_3_loss: 0.9029 - model_3_1_loss: 2.4039 - model_3_2_loss: 5.0991e-19 - model_3_3_loss: -18.6403


In [8]:
top_ans = generate_gauss(X, decoder, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3, s=100)

100/100

In [16]:
for i in range(len(top_ans)):
    top1_t = top_ans[i][0][0]
    top1_p = top_ans[i][0][1]
    top1_adds = top_ans[i][0][2:-1]
    top1_catalyst = top_ans[i][0][-1]

___

In [10]:
h_model, decoder = get_h_model(trained_emb_model_path='emb.h5', in_dim=500, out_dim=433, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3)

Note that input tensors are instantiated via `tensor = tf.keras.Input(shape)`.
The tensor that caused the issue was: input_8:0


In [21]:
h_model.fit([X, y], 
            [y[:, :3], y[:, 3:7], 
            y[:, 7:129+7], y[:, 129+7:]], 
            batch_size=3, epochs=1)

Train on 3 samples
3/3 [==============================] - 2s 584ms/sample - loss: -32.2061 - model_13_loss: 0.9613 - model_13_1_loss: 2.3602 - model_13_2_loss: 5.0991e-19 - model_13_3_loss: -35.5277


In [22]:
top_ans = generate_h(X, decoder, adds_dim=129, catalyst_dim=297, pressure_dim=4, temperature_dim=3, s=100)

100/100

In [23]:
for i in range(len(top_ans)):
    top1_t = top_ans[i][0][0]
    top1_p = top_ans[i][0][1]
    top1_adds = top_ans[i][0][2:-1]
    top1_catalyst = top_ans[i][0][-1]